# 📰 Klasifikasi Berita dengan Topic Modeling, Naïve Bayes, dan SVM
UTS - Analisis Teks & Machine Learning

Notebook ini melakukan klasifikasi berita menggunakan ekstraksi fitur **Topic Modeling (LDA)** dan model **Naïve Bayes** serta **SVM**.

## Tahapan:
1. Load & eksplorasi data
2. Preprocessing teks
3. Ekstraksi fitur dengan LDA (Topic Modeling)
4. Split data (train/test)
5. Klasifikasi dengan Naïve Bayes & SVM
6. Evaluasi hasil & interpretasi topik

In [4]:
import pandas as pd
import numpy as np
import re, string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

nltk.download('stopwords')

# Load dataset
df = pd.read_csv('datasets/Berita.csv')
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,No,judul,berita,tanggal,kategori,link
0,1,Airlangga Harap Kenaikan UMP Tingkatkan Daya B...,Menteri Koordinator (Menko) Bidang Perekonomia...,"Minggu, 01 Des 2024 23:40 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412012...
1,2,PT SIER Beri Penghargaan untuk 50 Tenant Terba...,"Dalam rangka memeriahkan hari jadi ke-50, PT S...","Minggu, 01 Des 2024 20:45 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412012...
2,3,Prabowo Bakal Bentuk Kementerian Penerimaan Ne...,Wacana Presiden Prabowo Subianto akan membentu...,"Minggu, 01 Des 2024 19:40 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412011...
3,4,Sinergi Kemenag & BPJS Ketenagakerjaan Lindung...,BPJS Ketenagakerjaan dan Kementerian Agama (Ke...,"Minggu, 01 Des 2024 19:03 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412011...
4,5,Pemerintah Segera Bentuk Satgas PHK Usai Tetap...,Pemerintah akan segera membentuk Satuan Tugas ...,"Minggu, 01 Des 2024 19:00 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412011...


## 🔧 Preprocessing Data Teks

In [5]:
# Cek kolom yang tersedia
df.info()

# Asumsi kolom teks bernama 'isi_berita' dan label 'kategori'
df = df[['berita', 'kategori']].dropna()

# Fungsi pembersihan teks
stop_words = set(stopwords.words('indonesian'))

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r'\d+', '', text)
    tokens = [w for w in text.split() if w not in stop_words]
    return ' '.join(tokens)

df['clean'] = df['berita'].apply(clean_text)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   No        1500 non-null   int64 
 1   judul     1500 non-null   object
 2   berita    1500 non-null   object
 3   tanggal   1500 non-null   object
 4   kategori  1500 non-null   object
 5   link      1500 non-null   object
dtypes: int64(1), object(5)
memory usage: 70.4+ KB


,berita,kategori,clean
0,Menteri Koordinator (Menko) Bidang Perekonomia...,Ekonomi,menteri koordinator menko bidang perekonomian ...
1,"Dalam rangka memeriahkan hari jadi ke-50, PT S...",Ekonomi,rangka memeriahkan pt surabaya industrial esta...
2,Wacana Presiden Prabowo Subianto akan membentu...,Ekonomi,wacana presiden prabowo subianto membentuk mem...
3,BPJS Ketenagakerjaan dan Kementerian Agama (Ke...,Ekonomi,bpjs ketenagakerjaan kementerian agama kemenag...
4,Pemerintah akan segera membentuk Satuan Tugas ...,Ekonomi,pemerintah membentuk satuan tugas pemutusan hu...


## 🔠 Ekstraksi Fitur dengan Topic Modeling (LDA)

In [6]:
# Ubah teks menjadi representasi bag-of-words
cv = CountVectorizer(max_features=2000)
X_bow = cv.fit_transform(df['clean'])

# LDA untuk ekstraksi topik
lda = LatentDirichletAllocation(n_components=6, random_state=42)
topic_features = lda.fit_transform(X_bow)
topic_features.shape

(1500, 6)

## 🔍 Split Data untuk Training dan Testing

In [7]:
X_train, X_test, y_train, y_test = train_test_split(topic_features, df['kategori'], test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((1200, 6), (300, 6))

## 🤖 Klasifikasi Menggunakan Naïve Bayes dan SVM

In [8]:
# Model Naïve Bayes
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)

# Model SVM
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

# Evaluasi
print('Naïve Bayes Accuracy:', accuracy_score(y_test, y_pred_nb))
print('SVM Accuracy:', accuracy_score(y_test, y_pred_svm))

Naïve Bayes Accuracy: 0.8133333333333334
SVM Accuracy: 0.8166666666666667


## 📊 Laporan Evaluasi

In [9]:
print('--- Naïve Bayes ---')
print(classification_report(y_test, y_pred_nb))

print('\n--- SVM ---')
print(classification_report(y_test, y_pred_svm))

--- Naïve Bayes ---
               precision    recall  f1-score   support

      Ekonomi       0.84      0.88      0.86        73
Internasional       0.94      0.58      0.72        84
     Nasional       0.59      0.84      0.69        70
     Olahraga       1.00      0.99      0.99        73

     accuracy                           0.81       300
    macro avg       0.84      0.82      0.82       300
 weighted avg       0.85      0.81      0.81       300


--- SVM ---
               precision    recall  f1-score   support

      Ekonomi       0.86      0.89      0.87        73
Internasional       0.96      0.57      0.72        84
     Nasional       0.59      0.86      0.70        70
     Olahraga       1.00      0.99      0.99        73

     accuracy                           0.82       300
    macro avg       0.85      0.83      0.82       300
 weighted avg       0.86      0.82      0.82       300



## 🧩 Interpretasi
- LDA menghasilkan distribusi topik untuk setiap dokumen.
- Fitur ini digunakan oleh Naïve Bayes dan SVM untuk klasifikasi.
- Bandingkan akurasi keduanya: umumnya **SVM** lebih tinggi, tapi **Naïve Bayes** lebih cepat.
- Kamu juga bisa melihat kata-kata dominan pada tiap topik dari LDA.

In [10]:
# Menampilkan kata-kata dominan tiap topik
terms = cv.get_feature_names_out()
for idx, topic in enumerate(lda.components_):
    print(f'Topik {idx}:')
    print([terms[i] for i in topic.argsort()[:-10 - 1:-1]])
    print()

Topik 0:
['korban', 'kebakaran', 'orang', 'warga', 'pesawat', 'air', 'banjir', 'jalan', 'rumah', 'kota']

Topik 1:
['israel', 'gaza', 'senjata', 'gencatan', 'hamas', 'palestina', 'al', 'trump', 'serangan', 'suriah']

Topik 2:
['laut', 'pagar', 'tangerang', 'nelayan', 'kkp', 'misterius', 'menteri', 'perikanan', 'sertifikat', 'pt']

Topik 3:
['negara', 'presiden', 'kpk', 'anggota', 'yoon', 'indonesia', 'hukum', 'video', 'tersangka', 'ketua']

Topik 4:
['rp', 'persen', 'program', 'prabowo', 'indonesia', 'pemerintah', 'juta', 'ppn', 'menteri', 'harga']

Topik 5:
['indonesia', 'timnas', 'pemain', 'piala', 'gol', 'menit', 'pelatih', 'laga', 'tim', 'pertandingan']

